# 1 Author

**Student Name**:  Shivam Tyagi

**Student ID**:  220428206




# 2 Problem formulation

Building a Machine Learning model which aims at predictng the exact indoor spot of a particular location.
To clearly predict the exact indoor spot of the possible 2-3 spots per location, various features that an audio data posesses are explored and the models efficiency is tested using the acuuracy score along with confusion matrix.

It is interesting because different indoors can have different type of noise levels which result in different audio features, for example campus curve would have more noise as compared to campus reception. Thus by using the location along with audio features we can train the classifier so that it can predict the spot.

# 3 Machine Learning pipeline

1. **Data Load**: Loading the data files in .wav format from personal google drive.

2. **Label Conversion**:  Converted the labels as 1 ,2 ,3 for the three types of indoor spots available for all the locations except southbank where only 2 indoor locations were present. 

3. **Feature Extraction**: Specific spectral features related to audio files such as: centroid, bandwidth, mfcc etc were extracted.

4. **Component Analysis**: Following feature extraction, Principal component analysis was performed to take out only the most contributing features(3 in our case excluding the location, hence 4 input features).

5. **Data split**: The whole dataset is then split into training and validation with 70:30 raito.

6. **Standardisation**: The training and validation data is then normalized using mean and standard deviation, X = (X-mean)/sd.

7. **ML Model**: Different ML models like SVC, RandomForest are trained and the best is selected according to point number 8, validation scores. 

8. **Validations**: Perfromance of the model is then judged on the basis of accuracy,precision, recall,f1 score and confusion matrix. All of the above are calculated for the training as well as validation datset. 


In [1]:
from google.colab import drive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier  


In [10]:
# Run only to verify
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Run only to verify
directory_to_extract_to1 = '/content/drive/MyDrive/Data/MiniProject/sample1/'
directory_to_extract_to2 = '/content/drive/MyDrive/Data/MiniProject/sample2/'
directory_to_extract_to3 = '/content/drive/MyDrive/Data/MiniProject/sample3/'
directory_to_extract_to4 = '/content/drive/MyDrive/Data/MiniProject/sample4/'
directory_to_extract_to5 = '/content/drive/MyDrive/Data/MiniProject/sample5/'

zip_path=[]
zip_path1 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_1.zip'
zip_path2 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_2.zip'
zip_path3 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_3.zip'
zip_path4 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_4.zip'
zip_path5 = '/content/drive/MyDrive/Data/MiniProject/MLEndLS_5.zip'


with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to1)

with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to2)

with zipfile.ZipFile(zip_path3, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to3)

with zipfile.ZipFile(zip_path4, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to4)

with zipfile.ZipFile(zip_path5, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to5)

sample_path1 = '/content/drive/MyDrive/Data/MiniProject/sample1/*.wav'
sample_path2 = '/content/drive/MyDrive/Data/MiniProject/sample2/*.wav'
sample_path3 = '/content/drive/MyDrive/Data/MiniProject/sample3/*.wav'
sample_path4 = '/content/drive/MyDrive/Data/MiniProject/sample4/*.wav'
sample_path5 = '/content/drive/MyDrive/Data/MiniProject/sample5/*.wav'
files1 = glob.glob(sample_path1)
files2 = glob.glob(sample_path2)
files3 = glob.glob(sample_path3)
files4 = glob.glob(sample_path4)
files5 = glob.glob(sample_path5)



In [13]:
# Run only to verify
#Taking only the indoor spots from the whole csv
MLENDLS_df = pd.read_csv('MLEndLS.csv').set_index('file_id') 
for i in range(0,len(MLENDLS_df)):
  if MLENDLS_df['area'][i]=="british":
    MLENDLS_df['area'][i]=1
    if MLENDLS_df['spot'][i]=="greatcourt":
      MLENDLS_df['spot'][i]=1
    elif MLENDLS_df['spot'][i]=="room12":
      MLENDLS_df['spot'][i]=2
    elif MLENDLS_df['spot'][i]=="room23" or MLENDLS_df['spot'][i]=="room13":
      MLENDLS_df['spot'][i]=3
    else:
      MLENDLS_df['spot'][i]='Nan'
  elif MLENDLS_df['area'][i]=="campus":
    MLENDLS_df['area'][i]=2
    if MLENDLS_df['spot'][i]=="reception":
      MLENDLS_df['spot'][i]=1
    elif MLENDLS_df['spot'][i]=="curve":
      MLENDLS_df['spot'][i]=2
    elif MLENDLS_df['spot'][i]=="ground":
      MLENDLS_df['spot'][i]=3
    else:
      MLENDLS_df['spot'][i]='Nan'
  elif MLENDLS_df['area'][i]=="westend":
    MLENDLS_df['area'][i]=3
    if MLENDLS_df['spot'][i]=="national":
      MLENDLS_df['spot'][i]=1
    elif MLENDLS_df['spot'][i]=="market":
      MLENDLS_df['spot'][i]=2
    elif MLENDLS_df['spot'][i]=="charing":
      MLENDLS_df['spot'][i]=3
    else: 
      MLENDLS_df['spot'][i]='Nan'
  elif MLENDLS_df['area'][i]=="southbank":
    MLENDLS_df['area'][i]=4
    if MLENDLS_df['spot'][i]=="royal":
      MLENDLS_df['spot'][i]=1
    elif MLENDLS_df['spot'][i]=="waterloo":
      MLENDLS_df['spot'][i]=2
    else:
      MLENDLS_df['spot'][i]='Nan'
  elif MLENDLS_df['area'][i]=="kensington":
    MLENDLS_df['area'][i]=5
    if MLENDLS_df['spot'][i]=="dinosaur":
      MLENDLS_df['spot'][i]=1
    elif MLENDLS_df['spot'][i]=="hintze":
      MLENDLS_df['spot'][i]=2
    elif MLENDLS_df['spot'][i]=="marine":
      MLENDLS_df['spot'][i]=3
    else: 
      MLENDLS_df['spot'][i]='Nan'
  elif MLENDLS_df['area'][i]=="Euston":
    MLENDLS_df['area'][i]=6
    if MLENDLS_df['spot'][i]=="upper":
      MLENDLS_df['spot'][i]=1
    elif MLENDLS_df['spot'][i]=="library":
      MLENDLS_df['spot'][i]=2
    elif MLENDLS_df['spot'][i]=="ritblat":
      MLENDLS_df['spot'][i]=3
    else: 
      MLENDLS_df['spot'][i]='Nan'

df=MLENDLS_df.drop(MLENDLS_df[MLENDLS_df['spot'] == "Nan"].index)

# 4 Transformation stage

Three types of transformations have been done:
1. **Output Label Conversions**:

*   As we are interested in checking that if the audio was recorded at a particular spot of a location hence we have three classes to predict per location because for all the locations we have 3 different indoor spots except southbank where we have 2 of indoor sports.

*   We assign the value of labels as 1,2,3 for all the indoor spots for all location and we assign 1,2 for the indoor spots of southbank, by doing so we have converted the labels from categorical data to numeric data and hence easy to understand for most of the ML models.

2. **Feature Extraction**:

*   Different indoor places will definitely have different noise levels, for example campus curve would be noisier than campus reception hence spectral features like bandwidth, centroid, flatness and zero crossing rates are chosen which will have a higher value for for certain spots(noisier) as compared to other quiet places.

*   MFCC is used for audio similarity measures which will give similar outputs for similar type of audios.


3. **Input Feature Conversions**:


*   As all the features exctracted are in the form of numpy array hence  taking their mean and then they are given as input lables which would then be used as inputs to the classifier.
*   Not all the features would be contributing as desired, hence Principal Component Analysis is performed on these input labels, value of the number of components is increased until the highest accuracy is acheived.
*   PCA with number of components =3 gives the highest accuracy.

*   Also as we need to predict the spot of a particular location hence we need to keep the location column which has values from 1 to 6. So PCA is performed on all other columns and 3 best are selected as they gave higher accuracy.
*   So finally we have 4 input features which we use as the inputs to the classifier, one is location and the rest 3 are from PCA.











In [14]:
# Run only to verify
def getPitch(x,fs,winLen=0.02):
  #winLen = 0.02 
  p = winLen*fs
  frame_length = int(2**int(p-1).bit_length())
  hop_length = frame_length//2
  f0, voiced_flag, voiced_probs = librosa.pyin(y=x, fmin=80, fmax=450, sr=fs,
                                                 frame_length=frame_length,hop_length=hop_length)
  return f0,voiced_flag

In [15]:
# Run only to verify
def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
  X,y =[],[]
  for file in tqdm(files):
    fileID = file.split('/')[-1]
    file_name = file.split('/')[-1]
    if fileID in labels_file.index: #Using only the files with indoor spots according to the csv which now have only indoor spots
      yi = labels_file.loc[fileID]['spot']

      fs = None # if None, fs would be 22050
      x, fs = librosa.load(file,sr=fs)
      mfcc = librosa.feature.mfcc(y = x, sr = fs, hop_length=513, n_mfcc=13) #for audio similarity measures
      cent = librosa.feature.spectral_centroid(y=x, sr=fs)
      spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=fs)
      S = np.abs(librosa.stft(x))
      contrast = librosa.feature.spectral_contrast(S=S, sr=fs)
      flatness = librosa.feature.spectral_flatness(y=x)
      zrcr=librosa.feature.zero_crossing_rate(x)
    
      if scale_audio: x = x/np.max(np.abs(x))
      f0, voiced_flag = getPitch(x,fs,winLen=0.02)
      
      
      power = np.sum(x**2)/len(x)
      pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
      pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
      voiced_fr = np.mean(voiced_flag)

      xi = [labels_file.loc[fileID]['area'],power,pitch_mean,pitch_std,voiced_fr,np.mean(mfcc),np.mean(cent),np.mean(spec_bw),np.mean(contrast),np.mean(flatness),np.mean(zrcr)]
      X.append(xi)
      y.append(yi)

  return np.array(X),np.array(y)

In [ ]:
# Run only to verify
X1,y1 = getXy(files1, labels_file=df, scale_audio=True, onlySingleDigit=True)
X2,y2 = getXy(files2, labels_file=df, scale_audio=True, onlySingleDigit=True)
X3,y3 = getXy(files3, labels_file=df, scale_audio=True, onlySingleDigit=True)
X4,y4 = getXy(files4, labels_file=df, scale_audio=True, onlySingleDigit=True)
X5,y5 = getXy(files5, labels_file=df, scale_audio=True, onlySingleDigit=True)

X=np.append(X1,X2,axis=0)
X=np.append(X,X3,axis=0)
X=np.append(X,X4,axis=0)
X=np.append(X,X5,axis=0)

y=np.append(y1,y2,axis=0)
y=np.append(y,y3,axis=0)
y=np.append(y,y4,axis=0)
y=np.append(y,y5,axis=0)

ys=pd.DataFrame(y)
ys.to_csv("Label.csv")

Z=pd.DataFrame(X)
Z.to_csv("Features.csv")

In [2]:
# Start from here after importing libraries, as up till now after extracting the features the csv is converetd and i am providing the same
Z=pd.read_csv("Features_advanced.csv")
Z.drop(Z.columns[0], axis=1, inplace=True)

ar=np.array(Z['0'])
ar.ravel()
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scalar.fit(Z.iloc[ : , 1:])
scaled_data = scalar.transform(Z.iloc[ : , 1:])

from sklearn.decomposition import PCA
pca = PCA(n_components = 3)
pca.fit(scaled_data)

x_pca = pca.transform(scaled_data)

X=np.array(x_pca)
X=np.column_stack((X,ar))
ys=pd.read_csv("Label_advanced.csv")
ys.drop(ys.columns[0], axis=1, inplace=True)
y=np.array(ys)

print(X.shape)
print(y.shape)
y=y.ravel()

(1176, 4)
(1176, 1)


# 5 Modelling

1. **SVM**: SVM SVC classifier is trained because it is majorly used for the data which does not have a known distribution i.e., audio data in our case. Also it does not suffer overfitting and have very effect of outliers which may be some audios with huge amount of noise indoors at a spot at a different point of time and hence could have caused issues in prediction. It gave the best results with cost parameter =2.

2. **Random Forest Classifier**: Random Forest Classifier was trained as it handles non-linearity of parameters effeiciently, and also like SVM it is also robust to outlier. But it is not chosen because it was overfitting with incresing n_estimators used(10,20,30) on the training dataset, without any significant change in the validation accuracy, whcih can be checked if "Rand" is given as choice in below code.

3. **KNeighbour Classifier**: It was trained as through KNN new data can be added seamlessly which will not impact the accuracy of the algorithm. The KNN model has ben used with n_neighbors=[5, 10, 50]. As 70% of the data (800+) is used for training the data, so about 274 data rows were present for each class while training. But it was not chosen because of lower accuracy than SVM.

At last SVM was chosen because of higher accuracy on validation datset with cost parameter value as 2 and without any overfitting on the training dataset.


In [3]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3,random_state=10)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

choice="Svc"

if choice=="Rand":
  print("RandomForest:")
  model  = RandomForestClassifier(n_estimators=10)
elif choice=="Svc":
  print("SVC:")
  model  = svm.SVC(C=2)
else:
    print("KNN")
    model=KNeighborsClassifier(n_neighbors=50) 

model.fit(X_train,y_train)



SVC:


SVC(C=2)

# 6 Methodology

The model training is already explianed in the previous section.

Performance of the modles were accessed through the following:

1. Accuracy: Accuracy is the ratio of correct predictions out of all predictions made by an algorithm. It can be calculated by dividing precision by recall or as 1 minus false negative rate (FNR) divided by false positive rate (FPR).
2. Precision: The Precision is the ratio of true positives over the sum of false positives and true negatives. It is also known as positive predictive value.
3. Recall: Recall is the ratio of correctly predicted outcomes to all predictions. It is also known as sensitivity or specificity.
4. f1 score: The F1-score combines these three metrics into one single metric that ranges from 0 to 1 and it takes into account both Precision and Recall.
5. Confusion matrix: This matrix tell us all the correctly clasified along with incorrectly classified labels in the form of a matrix hence easier to read.

All of the above were kept into account while chosing the final classifier and their results were compared. SVM got the highest score for them all and hence was chose to finally train the data.

# 7 Dataset

1. 2487 audio files have been used in total out of which 1176 are outdoors and 1311 are indoors.
2. Input features and output label conversion is done for all the files already explained earlier which is then followed by PCA to select only relevant input features.
3. The total files are then divided into 70:30 raito for training and test data split. 
4. The 70% data is then used to train the classifier and the remaining 30% used to validate the classifier.

All the code for feature extraction, label conversion and data split is already done in above sections and explained.


# 8 Results



In [8]:
yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

pt=precision_score(yt_p,y_train,average="weighted")
rt=recall_score(yt_p,y_train,average="weighted")
f1t=f1_score(yt_p,y_train,average="weighted")
mt=metrics.confusion_matrix(yt_p,y_train)


pv=precision_score(yv_p,y_val,average="weighted")
rv=recall_score(yv_p,y_val,average="weighted")
f1v=f1_score(yv_p,y_val,average="weighted")
mv=metrics.confusion_matrix(yv_p,y_val)

print('Training Accuracy', np.mean(yt_p==y_train),"\nTraining precision",pt,"\nTraining recall",rt,"\nTrianing f1 score",f1t)
print("Confusion Matrix for training period:",mt,"\n")

print('Validation  Accuracy', np.mean(yv_p==y_val),"\nValidation precision",pv,"\nValidation recall",rv,"\nValidation f1 score",f1v)
print("Confusion Matrix for validation period:",mv)

Training Accuracy 0.6646415552855407 
Training precision 0.7459384881605963 
Training recall 0.6646415552855407 
Trianing f1 score 0.6869594271173859
Confusion Matrix for training period: [[238  51  41]
 [ 54 246 111]
 [  7  12  63]] 

Validation  Accuracy 0.5694050991501416 
Validation precision 0.645318922087578 
Validation recall 0.5694050991501416 
Validation f1 score 0.596360867531298
Confusion Matrix for validation period: [[97 18 21]
 [23 87 60]
 [ 9 21 17]]


In [9]:
mean = np.mean(X_train,axis=0)
sd=np.std(X_train,axis=0)
X_train = (X_train-mean)/sd
X_val  = (X_val-mean)/sd

choice="Svc"
if choice=="Rand":
  print("RandomForest:")
  model  = RandomForestClassifier()
elif choice=="Svc":
  print("SVC:")
  model  = svm.SVC(C=2)
elif choice=="Log": 
  print("Logistic Regression Accuracy:")
  model  = LogisticRegression()

model.fit(X_train,y_train)


yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

pt=precision_score(yt_p,y_train,average="weighted")
rt=recall_score(yt_p,y_train,average="weighted")
f1t=f1_score(yt_p,y_train,average="weighted")
mt=metrics.confusion_matrix(yt_p,y_train)


pv=precision_score(yv_p,y_val,average="weighted")
rv=recall_score(yv_p,y_val,average="weighted")
f1v=f1_score(yv_p,y_val,average="weighted")
mv=metrics.confusion_matrix(yv_p,y_val)

print('Training Accuracy', np.mean(yt_p==y_train),"\nTraining precision",pt,"\nTraining recall",rt,"\nTrianing f1 score",f1t)
print("Confusion Matrix for training period:",mt,"\n")

print('Validation  Accuracy', np.mean(yv_p==y_val),"\nValidation precision",pv,"\nValidation recall",rv,"\nValidation f1 score",f1v)
print("Confusion Matrix for validation period:",mv)

SVC:
Training Accuracy 0.6646415552855407 
Training precision 0.7459384881605963 
Training recall 0.6646415552855407 
Trianing f1 score 0.6869594271173859
Confusion Matrix for training period: [[238  51  41]
 [ 54 246 111]
 [  7  12  63]] 

Validation  Accuracy 0.5694050991501416 
Validation precision 0.645318922087578 
Validation recall 0.5694050991501416 
Validation f1 score 0.596360867531298
Confusion Matrix for validation period: [[97 18 21]
 [23 87 60]
 [ 9 21 17]]


# 9 Conclusions

After training different models and comparing their effeciency through the defined metrices SVM was found out to be the best.

Classes 1 and 2 were predicted correctly more number of time, class 2 and 3 shows some confusion among themselves which means that these indoor spots have less difference in noise.

The confusion between class 1 and class 3 of all the locations is high, which mean that these spots have more difference in noise than between any other combination.

The scores of the best model i.e., SVM for predicting an audio as indoors or outdoors are as follows:

SVC:

Training Accuracy 0.6646415552855407 

Training precision 0.7459384881605963 

Training recall 0.6646415552855407 

Trianing f1 score 0.6869594271173859

Confusion Matrix for training period: 

[[238  51  41]

[ 54 246 111]

[  7  12  63]] 

Validation  Accuracy 0.5694050991501416 

Validation precision 0.645318922087578 

Validation recall 0.5694050991501416 

Validation f1 score 0.596360867531298

Confusion Matrix for validation period: 

[[97 18 21]

[23 87 60]
 
[ 9 21 17]]